In [1]:
import emcee
from scipy.optimize import minimize
import corner
import ptemcee
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from emukit.core import ParameterSpace, ContinuousParameter
from GPy.models import GPRegression
from emukit.model_wrappers import GPyMultiOutputWrapper
from GPy.kern import RBF

1-dimensional input, 2-dimensional outputs

In [2]:
parameter_space = ParameterSpace([ContinuousParameter('x', 0, 1)])

In [3]:
from emukit.core.initial_designs import RandomDesign

design = RandomDesign(parameter_space)
num_data_points = 2
X = design.get_samples(num_data_points)

In [4]:
def f(X): # add noise
    y1 = np.sin(X)
    y2 = np.cos(X)
    return np.hstack([np.sin(X),np.cos(X)])

In [5]:
Y = f(X)
kernel = RBF(input_dim=1)
model_gpy = GPRegression(X,Y,kernel=kernel, noise_var=0.01**2)
model_emukit = GPyMultiOutputWrapper(model_gpy, n_outputs=2, n_optimization_restarts=1)

Predict 1 point:

In [6]:
model_emukit.predict(np.array(0.5).reshape(1,-1))
# Only 1 variance? Covariance?

(array([[0.47537514, 0.87433045]]), array([[0.00411324]]))

In [7]:
model_emukit.predict_covariance(np.array(0.5).reshape(1,-1)) 
# Is this the covariance estimate between 2 outputs? How to create a wrapper for 2 independent outputs?

array([[0.00411324]])

Use 0,1 to indicate which output to predict:

In [8]:
xtest = np.array([[0.5],[0.5]])
new_column = [[0], [1]]
xtest = np.append(xtest, new_column, axis=1)
xtest

array([[0.5, 0. ],
       [0.5, 1. ]])

In [9]:
model_emukit.predict(xtest)
# Same result?

(array([[0.47537514, 0.87433045],
        [0.47537514, 0.87433045]]),
 array([[0.00411324],
        [0.00411324]]))

TO UPDATE

In [10]:
from emukit.experimental_design.acquisitions.contour_MultiObs import contourWeightedEI

In [11]:
a_list = [0.2, 0.5]
alpha_list = [2, 2]
w_list = [0.5, 0.5]

In [12]:
contour_MD = contourWeightedEI(models = model_emukit, a_list = a_list, alpha_list = alpha_list, w_list = w_list)